In [1]:
import sqlparse

from django.db import connection, reset_queries

from wagtail.images.models import Image

from cast import appsettings as settings
from cast.renditions import Rectangle, Width, Height, RenditionFilters

In [2]:
post = Post.objects.get(slug="november-2023-11-13")

In [3]:
rendition = post.get_all_renditions().first()

In [4]:
rendition.filter_spec

'width-1110|format-avif'

In [5]:
renditions = post.get_all_renditions()

In [6]:
renditions.count()

238

In [7]:
Post.objects.get_images_for_posts(Post.objects.filter(slug="november-2023-11-13"))

[('image', <Image: Nora beim Kinderarzt>),
 ('gallery', <Image: Nora mit Laterne im Kinderwagen>),
 ('gallery', <Image: Der Kita-Martinsumzug geht gleich los>),
 ('gallery', <Image: Der Kita-Martinsumzug geht jetzt wirklich gleich los>),
 ('gallery', <Image: Nora mit Jasmin auf der Bank>),
 ('gallery', <Image: Claas Klasse geht zur Martinsfeier in der Schule>),
 ('gallery', <Image: Nora mit Laterne auf dem Schulhof>),
 ('gallery', <Image: Martinsfeier in der Schule>),
 ('gallery', <Image: Puppentheater am Staufenplatz>),
 ('gallery', <Image: Nora und Claas und das Neinhorn>),
 ('gallery', <Image: Nora Claas und das Neinhorn nochmal>),
 ('gallery',
  <Image: Claas, Leander und Carl klettern auf einer Kletterspinne>),
 ('gallery',
  <Image: Claas, Leander und Carl klettern immer noch auf der Kletterspinne>)]

In [8]:
images = Post.objects.get_images_for_posts(Post.objects.all())

In [9]:
len(images)

2006

# Get all filter strings for images

In [3]:
def count_queries(func):
    reset_queries()
    renditions = list(func())
    return len(connection.queries)


def show_queries(queries):
    for query in queries:
        formatted_sql = sqlparse.format(query['sql'], reindent=True, keyword_case='upper')
        print(formatted_sql)    


In [4]:
%%time
reset_queries()
posts_queryset = Post.objects.prefetch_related("images", "galleries__images")
images = []
for post in posts_queryset:
    for image_type, image in post.get_all_images():
        images.append((image_type, image))
print(len(connection.queries))
#show_queries(connection.queries)

4
CPU times: user 85.3 ms, sys: 8.21 ms, total: 93.5 ms
Wall time: 111 ms


In [5]:
%%time
reset_queries()
images = Post.objects.get_images_for_posts(Post.objects.all())
print(len(connection.queries))
# show_queries(connection.queries)

4
CPU times: user 122 ms, sys: 16.3 ms, total: 139 ms
Wall time: 155 ms


In [13]:
[slot] = [Rectangle(Width(w), Height(h)) for w, h in settings.CAST_REGULAR_IMAGE_SLOT_DIMENSIONS]
image_formats = settings.CAST_IMAGE_FORMATS
all_filters = []
for image in post.images.all():
    rendition_filters = RenditionFilters.from_wagtail_image(image=image, slots=[slot], image_formats=image_formats)
    all_filters.extend(rendition_filters.filter_strings)

In [14]:
rendition_filter_strings

NameError: name 'rendition_filter_strings' is not defined

In [35]:
post.images.count()

1

In [36]:
Image.objects.count()

2033

In [37]:
Rendition.objects.filter(image__in=Image.objects.filter(gallery__in=post.galleries.all()).union(post.images.all()).values_list("pk", flat=True))

<QuerySet [<Rendition: Rendition object (30465)>, <Rendition: Rendition object (30466)>, <Rendition: Rendition object (30467)>, <Rendition: Rendition object (30468)>, <Rendition: Rendition object (30469)>, <Rendition: Rendition object (30470)>, <Rendition: Rendition object (30471)>, <Rendition: Rendition object (30472)>, <Rendition: Rendition object (30473)>, <Rendition: Rendition object (30474)>, <Rendition: Rendition object (30475)>, <Rendition: Rendition object (30476)>, <Rendition: Rendition object (30477)>, <Rendition: Rendition object (30478)>, <Rendition: Rendition object (30479)>, <Rendition: Rendition object (30480)>, <Rendition: Rendition object (30481)>, <Rendition: Rendition object (30482)>, <Rendition: Rendition object (30483)>, <Rendition: Rendition object (30484)>, '...(remaining elements truncated)...']>

In [38]:
images = []
qs = post.galleries.all().prefetch_related("images")
for gallery in qs:
    for image in gallery.images.all():
        images.append(image)

In [39]:
len(images)

12

In [40]:
reset_queries()
images = []
qs = post.galleries.all().prefetch_related("images")
for gallery in qs:
    for image in gallery.images.all():
        images.append(image)
print(len(connection.queries))
show_queries(connection.queries)

2
SELECT "cast_gallery"."id",
       "cast_gallery"."created",
       "cast_gallery"."modified"
FROM "cast_gallery"
INNER JOIN "cast_post_galleries" ON ("cast_gallery"."id" = "cast_post_galleries"."gallery_id")
WHERE "cast_post_galleries"."post_id" = 480
SELECT ("cast_gallery_images"."gallery_id") AS "_prefetch_related_val_gallery_id",
       "wagtailimages_image"."id",
       "wagtailimages_image"."collection_id",
       "wagtailimages_image"."title",
       "wagtailimages_image"."file",
       "wagtailimages_image"."width",
       "wagtailimages_image"."height",
       "wagtailimages_image"."created_at",
       "wagtailimages_image"."uploaded_by_user_id",
       "wagtailimages_image"."focal_point_x",
       "wagtailimages_image"."focal_point_y",
       "wagtailimages_image"."focal_point_width",
       "wagtailimages_image"."focal_point_height",
       "wagtailimages_image"."file_size",
       "wagtailimages_image"."file_hash"
FROM "wagtailimages_image"
INNER JOIN "cast_gallery_images

# Prefetch?

In [41]:
post.pk

480

In [42]:
from django.db.models import Prefetch

In [19]:
prefetch_direct_images = Prefetch("images", queryset=Image.objects.all())
prefetch_gallery_images = Prefetch("galleries__images", queryset=Image.objects.all())
posts = Post.objects.all().prefetch_related(prefetch_direct_images, prefetch_gallery_images)

In [20]:
def get_images_from_posts(posts):
    images = []
    for post in posts:
        for image in post.images.all():
            images.append(("image", image))
        for gallery in post.galleries.all():
            for image in gallery.images.all():
                images.append(("gallery", image))
    return images

In [21]:
%%time
reset_queries()
images = get_images_from_posts(posts)
print(len(connection.queries))
#show_queries(connection.queries)

4
CPU times: user 86.6 ms, sys: 5.82 ms, total: 92.4 ms
Wall time: 108 ms


# Improved version

In [25]:
prefetch_direct_images = Prefetch("images", queryset=Image.objects.all())
prefetch_gallery_images = Prefetch("galleries__images", queryset=Image.objects.all(), to_attr='gallery_images')
posts = Post.objects.all().prefetch_related(prefetch_direct_images, prefetch_gallery_images)

def get_images_from_posts(posts):
    images = []
    for post in posts:
        # Direct images
        for image in post.images.all():
            images.append(("image", image))

        # Gallery images (using to_attr)
        for gallery in post.galleries.all():
            for image in getattr(gallery, 'gallery_images', []):
                images.append(("gallery", image))

    return images

In [26]:
%%time
reset_queries()
images = get_images_from_posts(posts)
print(len(connection.queries))
#show_queries(connection.queries)

4
CPU times: user 66.6 ms, sys: 2.49 ms, total: 69.1 ms
Wall time: 79.9 ms


# Much simpler

In [28]:
def get_images_from_posts(posts):
    images = []
    for post in posts:
        for image in post.images.all():
            images.append(("image", image))
        for gallery in post.galleries.all():
            for image in gallery.images.all():
                images.append(("gallery", image))
    return images

In [29]:
posts = Post.objects.all().prefetch_related("galleries__images", "images")

In [30]:
%%time
reset_queries()
images = get_images_from_posts(posts)
print(len(connection.queries))
#show_queries(connection.queries)

4
CPU times: user 151 ms, sys: 4.45 ms, total: 156 ms
Wall time: 173 ms
